In [1]:
%%time
# -*- coding: utf-8 -*-
import pandas as pd
import numpy as np
import requests
import re
import sys
print(sys.executable)
from bs4 import BeautifulSoup
import logging
from requests import Request, Session
from requests.exceptions import ConnectionError, Timeout, TooManyRedirects
# from airflow.operators.python_operator import BranchPythonOperator
# from airflow.operators.bash_operator import BashOperator
from bs4 import BeautifulSoup
import requests
import json
from datetime import datetime, timedelta
# from airflow.contrib.hooks.aws_hook import AwsHook
# from airflow.models import Variable
# from airflow.hooks.S3_hook import S3Hook
# import boto3
import gzip
import os
from io import StringIO
import time
import shutil
from tqdm import tqdm
import time


### Use list ###
basic_date_list = []
basic_news_article = []
depth_date_list = []
depth_news_article = []
clear_word = []


### Crawler 1 ###
def basic_crawler():
    """
    Crawler(data gathering)
    """
    d = []
    a = []

    for i in tqdm(range(1,200), desc="Extracting Web page.."):
        url = 'https://www.coindeskkorea.com/news/articleList.html?page={}&total=6048&box_idxno=&view_type=sm'.format(i)
        resp = requests.get(url)
        soup = BeautifulSoup(resp.content, 'lxml')
        time.sleep(0.01)

        for j in range(1,21):
            date = soup.select('#user-container div.float-center.custom-m.mobile.template.list.max-width-1250 div.user-content section article div.article-list section div:nth-of-type({}) div.text-block div.list-dated'.format(j))
            article = soup.select(
            '#user-container div.float-center.custom-m.mobile.template.list.max-width-1250 div.user-content section article div.article-list section div:nth-of-type({}) div.text-block p a'.format(
                j))

            d.append(date)
            a.append(article)

    for _ in d:
        if _:
            basic_date_list.append(_)

    for _ in a:
        if _:
            basic_news_article.append(_)

    return basic_date_list, basic_news_article


### Crawler 2 ###
def depth_crawler():
    """
    Optimization basic_crawl data
    """
    # depth_news_article
    processing_news = []
    for i in tqdm(basic_news_article, desc="Extracting News url.."):
        url = 'https://www.coindeskkorea.com'+str(i).split(' ')[2].split('"')[1]

        processing_news.append(url)

    for i in tqdm(processing_news, desc="Crawling News article.."):
        resq = requests.get(i)
        soup = BeautifulSoup(resq.text,'lxml')
        n = str(soup.find_all('p')).split('<p><a href=')[0]

        depth_news_article.append(n)

    # depth_date_list
    for i in tqdm(basic_date_list, desc="Crawling News datetime.."):
        d = i[0].text.split(' ')[-2]

        depth_date_list.append(d)

    return depth_news_article, depth_date_list


### pre-processing ###
def processing():
    """
    Article pre-processing
    """
    process = []
    for i in tqdm(depth_news_article, desc="processing complie 1"):
        a = re.compile('[가-힣]+').findall(i)

        process.append(a)

    for i in tqdm(process, desc="processing complie 2"):
        a = ' '.join(i)
        b = re.compile('[제보-보내주세요]').sub(' ',a)
        # c = b.replace('도자료','')

        clear_word.append(' '+b)

    return clear_word


### dataframe ###
def dataframe():
    df_article = pd.DataFrame(clear_word, columns=['article'])
    df_date = pd.DataFrame(depth_date_list, columns=['created_date'])
    df_total = pd.concat([df_date, df_article], axis=1)
    df_total = df_total.groupby('created_date').sum(str(df_total['article']))
    print("Make DataFrame Structure")

    return df_total.to_csv('/Users/yoo/Data-dev/nlp/reference/DataFrame/article_comparison.csv', encoding='utf-8'), df_total.to_csv('/Users/yoo/Data-dev/nlp/dev/article_comparison.csv', encoding='utf-8')
    # return df_total


# ### Upload S3 bucket ###
# def DataFrameUploader(df_total):
#     s3_bucket = 'russo-mydata'
#     s3_key = 'article_{{ tomorrow_ds_nodash }}.csv'
#     s3_hook = S3Hook(bucket)
#
#     return df_total.to_csv



### main_compile ###
basic_crawler()
depth_crawler()
processing()
dataframe()
# DataFrameUploader(dataframe())
print("Done")

Extracting Web page..:   0%|          | 0/199 [00:00<?, ?it/s]

/Users/yoo/opt/anaconda3/bin/python


processing complie 2: 100%|██████████| 796/796 [00:00<00:00, 33390.33it/s]


Make DataFrame Structure
Done
CPU times: user 1min 36s, sys: 3.29 s, total: 1min 39s
Wall time: 21min 6s


In [4]:
%%time
article = pd.read_csv('/Users/yoo/Data-dev/nlp/dev/article.csv')
article

CPU times: user 14.3 ms, sys: 4.46 ms, total: 18.7 ms
Wall time: 16.6 ms


,created_date,article
0,2018-04-19,도자료는 도자료는 으로 도자료는
1,2018-04-20,도자료는 으로
2,2018-04-24,도자료는 도자료는 도자료는 으로
3,2018-04-25,도자료는
4,2018-04-26,도자료는 으로
...,...,...
327,2020-06-04,짧은뉴스 오늘 을 통해 국 외 업계 이모저모를 전달해드립니다 도자료는 코인데스크...
328,2020-06-09,지닥 데이빗 코인원 등 최근 카카오 계열 암호화폐인 클레이 를 원화마켓에 상장했던...
329,2020-06-10,미국 전역에서 조지 플로이드 씨 사망으로 촉발된 시위가 이어지면서 석금이 없어 ...
330,2020-06-13,김태원 글로스퍼 대표가 일 사망한 것으로 확인됐다 김 대표가 부회장을 맡고 있는 ...


In [ ]:
%%time

# Price crawler
# -*- coding: utf-8 -*-

#####################
### import module ###
#####################
from requests import Request, Session
from requests.exceptions import ConnectionError, Timeout, TooManyRedirects
from airflow.operators.python_operator import BranchPythonOperator
from airflow.operators.bash_operator import BashOperator
from bs4 import BeautifulSoup
import requests
import json
from datetime import datetime, timedelta
from airflow.contrib.hooks.aws_hook import AwsHook
from airflow.models import Variable
from airflow.hooks.S3_hook import S3Hook
import gzip
import logging
import os
from io import StringIO
import boto3

import pandas as pd
import numpy as np
import time
import sys
import logging
import os
import shutil
import re



#####################
### Pandas Option ###
#####################
pd.options.display.float_format = '{:.5f}'.format



############################
### S3 bucket Parameters ###
############################
s3_sensorflag_bucket = 'russo-mydata'
s3_conn_id = 'russo-mydata'
s3_bucket = 'russo-mydata'


#######################
### Bithumb Crawler ###
#######################
def bithumb_crawler():
    """
    Bithumb Crawl during 24H information
    """
    bithumb_url = 'https://api.bithumb.com/public/ticker/ALL_KRW'
    html = requests.get(bithumb_url)
    soup = BeautifulSoup(html.text, "html.parser")
    soup = soup.text.replace('{"status":"0000","data":{', '')

    currency = []
    timestamp_kst = []
    opening_price = []
    high_price = []
    low_price = []
    close_price = []
    acc_trade_value = []
    unit_traded = []

    for i in soup.split('},')[:-1]:
        currency.append(i.split(':')[0].replace('"', ''))
        timestamp_kst.append(datetime.utcfromtimestamp(
            int(soup.split('},')[-1].replace('"', '').replace('}}', '').replace('date:', '')) / 1000).strftime(
            '%Y-%m-%d'))
        opening_price.append(i.split(',')[0].split(':')[2].replace('"', ''))
        high_price.append(i.split(',')[3].split(':')[1].replace('"', ''))
        low_price.append(i.split(',')[2].split(':')[1].replace('"', ''))
        close_price.append(i.split(',')[1].split(':')[1].replace('"', ''))
        acc_trade_value.append(i.split(',')[8].split(':')[1].replace('"', ''))
        unit_traded.append(i.split(',')[7].split(':')[1].replace('"', ''))

    bithumb_df = pd.concat([pd.DataFrame(currency, columns=['currency'])
                               , pd.DataFrame(timestamp_kst, columns=['timestamp_kst'])
                               , pd.DataFrame(opening_price, columns=['opening_price'])
                               , pd.DataFrame(high_price, columns=['high_price'])
                               , pd.DataFrame(low_price, columns=['low_price'])
                               , pd.DataFrame(close_price, columns=['close_price'])
                               , pd.DataFrame(acc_trade_value, columns=['acc_trade_value'])
                               , pd.DataFrame(unit_traded, columns=['unit_traded'])]
                           , axis=1)

    bithumb_df['exchange'] = 'Bithumb'
    print("Bithumb Done!")

    return bithumb_df



#######################
### Coinone Crawler ###
#######################
def coinone_crawler():
    """
    Coinone Crawl during 24H information
    """
    coinone_url = 'https://api.coinone.co.kr/ticker?currency=all'
    html = requests.get(coinone_url)
    soup = BeautifulSoup(html.text, "html.parser")

    currency = []
    timestamp_kst = []
    opening_price = []
    high_price = []
    low_price = []
    close_price = []
    acc_trade_value = []
    unit_traded = []

    for i in soup.text.split(':{')[1:]:
        currency.append(i.split(',')[0].split(':')[1].replace('"', ''))
        timestamp_kst.append(datetime.utcfromtimestamp(
            int(soup.text.split(':{')[0].split(',')[2].split(':')[1].replace('"', ''))).strftime('%Y-%m-%d'))
        opening_price.append(i.split(',')[1].split(':')[1].replace('"', ''))
        high_price.append(i.split(',')[3].split(':')[1].replace('"', ''))
        low_price.append(i.split(',')[2].split(':')[1].replace('"', ''))
        close_price.append(i.split(',')[4].split(':')[1].replace('"', ''))
        acc_trade_value.append(float(i.split(',')[4].split(':')[1].replace('"', '')) * float(
            i.split(',')[5].split(':')[1].replace('"', '')))
        unit_traded.append(i.split(',')[5].split(':')[1].replace('"', ''))

    coinone_df = pd.concat([pd.DataFrame(currency, columns=['currency'])
                               , pd.DataFrame(timestamp_kst, columns=['timestamp_kst'])
                               , pd.DataFrame(opening_price, columns=['opening_price'])
                               , pd.DataFrame(high_price, columns=['high_price'])
                               , pd.DataFrame(low_price, columns=['low_price'])
                               , pd.DataFrame(close_price, columns=['close_price'])
                               , pd.DataFrame(acc_trade_value, columns=['acc_trade_value'])
                               , pd.DataFrame(unit_traded, columns=['unit_traded'])]
                           , axis=1)

    coinone_df['currency'] = [i.upper() for i in coinone_df['currency']]
    coinone_df['exchange'] = 'Coinone'
    print("Coinone Done!")

    return coinone_df



######################
### Korbit Crawler ###
######################
def korbit_crawler():
    """
    Korbit Crawl during 24H information
    """
    korbit_url = 'https://api.korbit.co.kr/v1/ticker/detailed/all'
    html = requests.get(korbit_url)
    soup = BeautifulSoup(html.text, "html.parser")

    currency = []
    timestamp_kst = []
    opening_price = []
    high_price = []
    low_price = []
    close_price = []
    acc_trade_value = []
    unit_traded = []

    for i in soup.text.split('},'):
        currency.append(i.split(':')[0].replace('{', '').replace('"', ''))
        timestamp_kst.append(datetime.utcfromtimestamp(int(i.split(':')[2].split(',')[0]) / 1000).strftime('%Y-%m-%d'))
        opening_price.append(i.split(',')[2].split(':')[1].replace('"', ''))
        high_price.append(i.split(',')[6].split(':')[1].replace('"', ''))
        low_price.append(i.split(',')[5].split(':')[1].replace('"', ''))
        close_price.append(i.split(',')[1].split(':')[1].replace('"', ''))
        acc_trade_value.append(float(i.split(',')[7].split(':')[1].replace('"', '')) * float(
            i.split(',')[1].split(':')[1].replace('"', '')))
        unit_traded.append(i.split(',')[7].split(':')[1].replace('"', ''))

    korbit_df = pd.concat([pd.DataFrame(currency, columns=['currency'])
                              , pd.DataFrame(timestamp_kst, columns=['timestamp_kst'])
                              , pd.DataFrame(opening_price, columns=['opening_price'])
                              , pd.DataFrame(high_price, columns=['high_price'])
                              , pd.DataFrame(low_price, columns=['low_price'])
                              , pd.DataFrame(close_price, columns=['close_price'])
                              , pd.DataFrame(acc_trade_value, columns=['acc_trade_value'])
                              , pd.DataFrame(unit_traded, columns=['unit_traded'])]
                          , axis=1)

    korbit_df['currency'] = [i.split('_')[0].upper() for i in korbit_df['currency']]
    korbit_df['exchange'] = 'Korbit'
    print("Korbit Done!")

    return korbit_df



#####################
### Gopax Crawler ###
#####################
def gopax_crawler():
    """
    Gopax Crawl during 24H information
    """
    gopax_url = 'https://api.gopax.co.kr/trading-pairs/stats'
    html = requests.get(gopax_url)
    soup = BeautifulSoup(html.text, "html.parser")

    currency = []
    timestamp_kst = []
    opening_price = []
    high_price = []
    low_price = []
    close_price = []
    acc_trade_value = []
    unit_traded = []

    for i in soup.text.split(',{'):
        currency.append(i.split(',')[0].split(':')[1].replace('"', ''))
        timestamp_kst.append(i.split(',')[6].split(':')[1].replace('"', '').replace('T', ' '))
        opening_price.append(i.split(',')[1].split(':')[1])
        high_price.append(i.split(',')[2].split(':')[1])
        low_price.append(i.split(',')[3].split(':')[1])
        close_price.append(i.split(',')[4].split(':')[1])
        acc_trade_value.append(float(i.split(',')[4].split(':')[1]) * float(i.split(',')[5].split(':')[1]))
        unit_traded.append(i.split(',')[5].split(':')[1])

    gopax_df = pd.concat([pd.DataFrame(currency, columns=['currency']),
                          pd.DataFrame(timestamp_kst, columns=['timestamp_kst']),
                          pd.DataFrame(opening_price, columns=['opening_price']),
                          pd.DataFrame(high_price, columns=['high_price']),
                          pd.DataFrame(low_price, columns=['low_price']),
                          pd.DataFrame(close_price, columns=['close_price']),
                          pd.DataFrame(acc_trade_value, columns=['acc_trade_value']),
                          pd.DataFrame(unit_traded, columns=['unit_traded'])],
                         axis=1)

    gopax_df['currency'] = [i.split('-')[0].upper() for i in gopax_df['currency']]
    gopax_df['exchange'] = 'Gopax'
    print("Gopax Done!")

    return gopax_df



#####################
### Upbit Crawler ###
#####################
def upbit_crawler():
    """
    Upbit Crawl during 24H information
    first, Crawling currency-pair
    second, Crawling 1H data
    third, Union 1H data(1H * 24)
    """
    # Crawling currency-pair
    upbit_url = 'https://api.upbit.com/v1/market/all'
    html = requests.get(upbit_url)
    soup = BeautifulSoup(html.text, "html.parser")

    upbit_currency = soup.text.split('},')

    ignore_char = '""'

    basis_dy = []
    exchange = []
    currency_pair = []
    currency_name = []

    for i in upbit_currency:
        currency_pair.append(i.split(':')[1].split(',')[0].replace('"', ''))
        currency_name.append(i.split(':')[3].replace('}]', '').replace('"', ''))

    currency_pair_df = pd.DataFrame(currency_pair)
    currency_name_df = pd.DataFrame(currency_name)

    upbit_pair = pd.concat([currency_pair_df, currency_name_df], axis=1)
    upbit_pair.columns = ['currency_pair', 'name']

    upbit_pair['currency_krw'] = [i.split('-')[0] for i in upbit_pair['currency_pair']]
    upbit_pair = upbit_pair.where(upbit_pair['currency_krw'] == 'KRW')
    upbit_pair = upbit_pair.dropna(axis=0)

    currency_m = []
    timestamp_kst_m = []
    opening_price_m = []
    high_price_m = []
    low_price_m = []
    close_price_m = []
    acc_trade_value_m = []
    unit_traded_m = []

    # crawling 1H data
    for i in upbit_pair['currency_pair']:
        upbit_price = 'https://api.upbit.com/v1/candles/minutes/60?market={}&count=24'.format(i)
        time.sleep(0.1)
        html = requests.get(upbit_price)
        soup = BeautifulSoup(html.text, 'html.parser')
        soup = soup.text.split(',{')

        for j in soup:
            currency_m.append(j.split(',')[0].split(':')[1].replace('"', ''))
            timestamp_kst_m.append(j.split(',')[1].split(':')[1].replace('T', ' ').replace('"', ''))
            opening_price_m.append(j.split(',')[3].split(':')[1])
            high_price_m.append(j.split(',')[4].split(':')[1])
            low_price_m.append(j.split(',')[5].split(':')[1])
            close_price_m.append(j.split(',')[6].split(':')[1])
            acc_trade_value_m.append(j.split(',')[8].split(':')[1])
            unit_traded_m.append(j.split(',')[9].split(':')[1])

    upbit_df_m = pd.concat([pd.DataFrame(currency_m, columns=['currency_m']),
                            pd.DataFrame(timestamp_kst_m, columns=['timestamp_kst_m']),
                            pd.DataFrame(opening_price_m, columns=['opening_price_m']),
                            pd.DataFrame(high_price_m, columns=['high_price_m']),
                            pd.DataFrame(low_price_m, columns=['low_price_m']),
                            pd.DataFrame(close_price_m, columns=['close_price_m']),
                            pd.DataFrame(acc_trade_value_m, columns=['acc_trade_value_m']),
                            pd.DataFrame(unit_traded_m, columns=['unit_traded_m'])],
                           axis=1)

    upbit_df_m['acc_trade_value_m'] = upbit_df_m['acc_trade_value_m'].astype(float)
    upbit_df_m['unit_traded_m'] = upbit_df_m['unit_traded_m'].astype(float)

    # Union 1H data
    currency = []
    timestamp_kst = []
    opening_price = []
    high_price = []
    low_price = []
    close_price = []
    acc_trade_value = []
    unit_traded = []

    currency.append(upbit_df_m.groupby('currency_m').max().reset_index(drop=False)['currency_m'])
    timestamp_kst.append(upbit_df_m.groupby('currency_m').max()['timestamp_kst_m'])
    opening_price.append(upbit_df_m.groupby('currency_m').min()['opening_price_m'])
    high_price.append(upbit_df_m.groupby('currency_m').max()['high_price_m'])
    low_price.append(upbit_df_m.groupby('currency_m').min()['low_price_m'])
    close_price.append(upbit_df_m.groupby('currency_m').max()['close_price_m'])
    acc_trade_value.append(upbit_df_m.groupby('currency_m').sum('acc_trade_value_m')['acc_trade_value_m'])
    unit_traded.append(upbit_df_m.groupby('currency_m').sum('unit_traded_m')['unit_traded_m'])

    upbit_df = pd.concat([pd.DataFrame(currency).T.set_index('currency_m'),
                          pd.DataFrame(timestamp_kst).T,
                          pd.DataFrame(opening_price).T,
                          pd.DataFrame(high_price).T,
                          pd.DataFrame(low_price).T,
                          pd.DataFrame(close_price).T,
                          pd.DataFrame(acc_trade_value).T,
                          pd.DataFrame(unit_traded).T],
                         axis=1)

    upbit_df = upbit_df.reset_index(drop=False)
    upbit_df.columns = ['currency', 'timestamp_kst', 'opening_price', 'high_price', 'low_price', 'close_price',
                        'acc_trade_value', 'unit_traded']

    upbit_df['currency'] = [i.split('-')[1] for i in upbit_df['currency']]
    upbit_df['exchange'] = 'Upbit'
    print("Upbit Done!")

    return upbit_df



#####################
### Summary Table ###
#####################
def MakeDataFrame(upbit_df, bithumb_df, coinone_df, korbit_df, gopax_df):
    """
    Make Total DataFrame
    """
    total_df = pd.concat([bithumb_df, coinone_df, korbit_df, upbit_df, gopax_df], axis=0)
    total_df.columns = ['base_currency', 'etime_date', 'open_price', 'high_price', 'low_price', 'close_price',
                        'krw_volume', 'base_volume', 'exchange']
    total_df = total_df[
        ['exchange', 'base_currency', 'open_price', 'high_price', 'low_price', 'close_price', 'krw_volume',
         'base_volume', 'etime_date']]
    total_df = total_df.reset_index(drop=True)
    print(total_df)
    print("All Done!")
    logging.info("-------------sys.argv list-------------", print(sys.argv), sys.argv)

    # return total_df # Original Method(On docker-compose airflow or local airflow)
    return total_df.to_csv('/Users/yoo/Data-dev/nlp/dev/price.csv') # Temporary save DataFrame



##############################
### Upload DataFrame to S3 ###
##############################
# def DataFrameToS3Upload(total_df):
#     """
#     Upload DataFrame(df) to S3 bucket
#     """
#     try:
#         result = total_df.to_dict()
#         results_string = str(total_df.T.to_dict().values()).replace("dict_values([", '').replace('])', '').replace("'",
#                                                                                                                    '"').replace(
#             ' ', '').replace('},{', '}{')
#         logging.info(results_string[:51])
#         logging.info('******************* Changing JSON FORMAT to Bytes *******************')
#         results_bytes = results_string.encode('utf-8')
#         logging.info('******************* gzip the results *******************')
#         results_gzip = gzip.compress(results_bytes)
#         results = results_gzip

#         dag_id = 'price_crawler_v0001'

#         s3_key = '/airflow/dags' + dag_id + str((datetime.today() + timedelta(days=-1)).strftime('%Y%m%d')).replace('-', '') + '.json.gzip'
#         s3_hook = AwsHook(aws_conn_id='{{ params.s3_conn_id }}')
#         s3_client_type = s3_hook.get_client_type(client_type='s3', region_name='ap-northeast-1')
#         s3 = s3_client_type
#         s3.put_object(
#             Bucket=s3_bucket,
#             Body=results,
#             Key=s3_key
#         )
#         print("--------------- Python process end ---------------")

#     except Exception as e:
#         print("S3 uploader Error", e)

#     return results


####################
### Main Complie ###
####################
df = pd.DataFrame(MakeDataFrame(
    upbit_df=upbit_crawler(),
    bithumb_df=bithumb_crawler(),
    coinone_df=coinone_crawler(),
    korbit_df=korbit_crawler(),
    gopax_df=gopax_crawler()))

print(df)

# DataFrameToS3Upload(df)
# s3_bucket = 'russo-mydata'
# s3_hook = S3Hook(s3_bucket)
#
#
# #S3 디렉토리 리스팅
# keys = s3_hook.list_keys(bucket_name=bucket)
#
# # s3에 업로드
# s3_key = 'airflow/price.csv'
# s3_hook.load_file(filename='/var/lib/airflow/test/test.csv',key=s3_key,bucket_name=s3_bucket)